#Base Model

In [ ]:
# 📦 Imports
import os
import gradio as gr
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader, WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 📄 Load documents: local + Wikipedia
local_loader = TextLoader("extracted_text.txt")  # 🔁 Replace with your actual file
wiki_loader = WikipediaLoader(query="New Delhi is the Capital of India", load_max_docs=2)

docs = local_loader.load() + wiki_loader.load()

# ✂️ Split documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

# 🔍 Embed and store in Chroma
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(split_docs, embedding_model)

# 🤖 Load local model with transformers
local_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256, truncation=True)
llm = HuggingFacePipeline(pipeline=local_pipeline)

# 🔗 Create RAG chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# 🧠 Define response function
def generate_response(question):
    try:
        response = rag_chain.invoke({"query": question})
        answer = response["result"]
        sources = response.get("source_documents", [])

        if not sources:
            return "**Answer (fallback):**\nThis question is outside the scope of the provided documents."

        source_text = "\n\n".join([doc.page_content for doc in sources])
        return f"**Answer:**\n{answer}\n\n**Sources:**\n{source_text}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

# 🎛️ Launch Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="markdown",
    title="🌐 RAG QA with Local + Wikipedia Knowledge",
    description="Ask questions based on local documents and Wikipedia. Powered by LangChain, Chroma, and Hugging Face's flan-t5-base running locally."
)

iface.launch(share=True, debug=True)